Aujourd'hui, je vais vous présenter mon premier travail sur Kaggle; au niveau de cette anayse je vais faire quelques critiques positives et négatives sur les différentes avis des visiteurs: Au niveau de ce travail, j'ai exploré les choses suivante: -

Le groupe d'âge de la femme qui a écrit le plus, le moins et très peu de critiques 

le type de produit acheté par chaque tranche d'age (noms de département,noms de classe ) 

Pour chaque département quel est le nombre ou la pourcentage de class name qu'il a

Pour chaque devision nom quel est le nombre departement name 

Pour chaque division name quel est le nombre des name classe 

Compter la fréquence des mots dans la colonne Texte de révision 

Wordcloud de la colonne Review Text 

Colonne revision text - Combien de critiques positives, négatives et neutres sont basées sur le sentiment et la valeur de polarité. 

Wordcloud de la colonne positive Review Text 

Wordcloud de la colonne de texte de révision négatif 

Utiliser l’algorithme multinomial naïf Bayes pour prédire quel produit a la note 5 et quel a la note1? 

Utilisation de l'algorithme multinomial Naive Bayes pour prédire quel produit est recommandé et lequel ne l'est pas?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
import itertools
warnings.filterwarnings('ignore')
review =pd.read_csv("../input/Womens-Clothing-E-Commerce-Reviews.csv")

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_rough=pd.read_csv("../input/Womens-Clothing-E-Commerce-Reviews.csv",index_col=False)
column_contain=['Clothing ID','Age','Title','Review Text','Rating','Recommended IND','Positive Feedback Count','Division Name','Department Name','Class Name']

  Maintenant, je vais savoir quelques informtions sur les données comme les types d'index et des colonnes, les valeurs non null et l'utilisation de la mémoire:

In [ ]:
df=pd.DataFrame(data=df_rough,columns=column_contain)
df.info()

On remarque que ces données contient 23486 entrées et 9 colonnes. Certaines des entrées manquent, comme Titre, Nom de la division, Nom du département et Nom de la classe.

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.fillna(0,inplace=True)

In [ ]:
for col in df.columns:
    print('{} unique element: {}'.format(col,df[col].nunique()))

In [ ]:
df.head()

Maintenat, on va donner un sens visuel des donées: on va colorier les données en fonction de la colonne "Rating"

In [ ]:
# Drop rows with missing values and drop duplicate
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

# Visualize pairplot of df
sns.pairplot(df, hue='Rating');

Maintenant on va voir quelques attributs en fonction des autres:

In [ ]:
df1=df['Rating'].value_counts().to_frame()
avgdf1 = df.groupby('Class Name').agg({'Rating': np.average})
avgdf2 = df.groupby('Class Name').agg({'Age': np.average})
avgdf3 = df.groupby('Rating').agg({'Age': np.average})

trace1 = go.Bar(
    x=avgdf1.index,
    y=round(avgdf1['Rating'],2),
    marker=dict(
        color=avgdf1['Rating'],
        colorscale = 'RdBu')
)

trace2 = go.Bar(
    x=df1.index,
    y=df1.Rating,
    marker=dict(
        color=df1['Rating'],
        colorscale = 'RdBu')
)

trace3 = go.Bar(
    x=avgdf2.index,
    y=round(avgdf2['Age'],2),
    marker=dict(
        color=avgdf2['Age'],
        colorscale = 'RdBu')
)

trace4 = go.Bar(
    x=avgdf3.index,
    y=round(avgdf3['Age'],2),
    marker=dict(
        color=avgdf3['Age'],
        colorscale = 'Reds')
)

fig = tools.make_subplots(rows=2, cols=2)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout']['xaxis1'].update(title='Class')
fig['layout']['yaxis1'].update(title='Average Rating')
fig['layout']['xaxis2'].update(title='Rating')
fig['layout']['yaxis2'].update(title='Count')
fig['layout']['xaxis3'].update(title='Class')
fig['layout']['yaxis3'].update(title='Average Age of the Reviewers')
fig['layout']['xaxis4'].update(title='Rating')
fig['layout']['yaxis4'].update(title='Average Age of the Reviewers')

fig['layout'].update(height=800, width=900,showlegend=False)
py.iplot(fig)

In [ ]:
cv = df['Class Name'].value_counts()

trace = go.Scatter3d( x = avgdf1.index,
                      y = avgdf1['Rating'],
                      z = cv[avgdf1.index],
                      mode = 'markers',
                      marker = dict(size=10,color=avgdf1['Rating']),
                      hoverinfo ="text",
                      text="Class: "+avgdf1.index+" \ Average Rating: "+round(avgdf1['Rating'],2).apply(str)+" \ Number of Reviewers: "+cv[avgdf1.index].apply(str)
                      )

data = [trace]
layout = go.Layout(title="Average Rating & Class & Number of Reviewers",
                   scene = dict(
                    xaxis = dict(title='Class'),
                    yaxis = dict(title='Average Rating'),
                    zaxis = dict(title='Number of Sales'),),
                   margin = dict(l=30, r=30, b=30, t=30))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)
plt.savefig('3D_Scatter.png')

on a remarqué qu'il y a des texte de revue qui sont long et d'autres très courts Alors il y a-t-il un corrélation entre la longueure des commentaires écrites par les critqueurs et la note qu'ils donnent?

In [ ]:
df['Review Text']=df['Review Text'].astype(str)
df['Review Length']=df['Review Text'].apply(len)

In [ ]:
g = sns.FacetGrid(data=df, col='Rating')
g.map(plt.hist, 'Review Length', bins=50)

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(x='Rating', y='Review Length', data=df)

De la boîte à moustaches ci-dessus, nous pouvons conclure que les évaluations 3 et 4 ont plus de lenteur en revue.

In [ ]:
rating = df.groupby('Rating').mean()
rating.corr()

In [ ]:
sns.heatmap(data=rating.corr(), annot=True)

La carte de corrélation ci-dessus montre qu'il n'y a pas beaucoup de corrélation entre les colonnes. Les colonnes telles que Review Length et positive Feedback Count sont légèrement corrélées. Et le nombre 0,93 en négatif indique qu'il n'y a aucune corrélation entre positive Feedback count et age. À mesure que l'âge augmente, la durée de l'examen diminue.

In [ ]:
df.head()

On va analyser les différentes ages afin de voir la tranche d'age qui domine:

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Distribution of age")
ax = sns.distplot(df["Age"], color = 'm')

In [ ]:
df.groupby(['Rating', pd.cut(df['Age'], np.arange(0,100,10))])\
       .size()\
       .unstack(0)\
       .plot.bar(stacked=True)

Sur le graphique à barres ci-dessus, nous pouvons dire que le groupe d’âge des 10 à 20 ans a attribué une note moins élevée. Il est évident. Dans ce groupe d'âge, les adolescents ne se soucient généralement pas des achats en ligne et des critiques. Le groupe d'âge 20-40 ont donné le plus 5 points par rapport à tous les autres groupes d'âge. En fait, c’est le groupe d’âge qui a attribué le plus grand nombre de critiques et d’évaluations. De même, le groupe d’âge au-dessus de 70 ans ne s’intéressait pas aux achats en ligne.

In [ ]:
plt.figure(figsize=(15,15))
df.groupby(['Department Name', pd.cut(df['Age'], np.arange(0,100,10))])\
       .size()\
       .unstack(0)\
       .plot.bar(stacked=True)

Dans le graphique ci-dessus, je veux me concentrer sur le département et le groupe d'âge pour vor chaque trange d'age par quoi elle est interssées. Les femmes âgées de 20 à 60 ans étaient plus actives et achetaient le produit en ligne. À partir du diagramme en barres ci-dessus, nous pouvons conclure que toutes les femmes étaient plus concentrées sur le département Tops et Dressess. Et, un peu concentré sur bottoms aussi mais pas tant que ça. Ils étaient moins concentrés sur le département Trend.

In [ ]:
plt.figure(figsize=(15,15))
df.groupby(['Class Name', pd.cut(df['Age'], np.arange(0,100,10))])\
       .size()\
       .unstack(0)\
       .plot.bar(stacked=True)

In [ ]:
z=df.groupby(by=['Department Name'],as_index=False).count().sort_values(by='Class Name',ascending=False)

plt.figure(figsize=(10,10))
sns.set_style("whitegrid")
ax = sns.barplot(x=z['Department Name'],y=z['Class Name'], data=z)
plt.xlabel("Department Name")
plt.ylabel("Count")
plt.title("Counts Vs Department Name")

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_curve, auc
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
%matplotlib inline
 
df1 = pd.read_csv("../input/Womens-Clothing-E-Commerce-Reviews.csv")
df = df1[['Review Text','Rating','Class Name','Age']]
#df1.info()
#df1.describe()
df1.head()


In [ ]:
df['Review Text'] = df['Review Text'].fillna('')
vectorizer = CountVectorizer()
analyzer = vectorizer.build_analyzer()

def wordcounts(s):
    c = {}
    if analyzer(s):
        d = {}
        w = vectorizer.fit_transform([s]).toarray()
        vc = vectorizer.vocabulary_
        for k,v in vc.items():
            d[v]=k
        for index,i in enumerate(w[0]):
            c[d[index]] = i
    return  c

df['Word Counts'] = df['Review Text'].apply(wordcounts)
df.head()

In [ ]:
selectedwords = ['awesome','great','fantastic','extraordinary','amazing','super',
                 'magnificent','stunning','impressive','wonderful','breathtaking',
                 'love','content','pleased','happy','glad','satisfied','lucky',
                 'shocking','cheerful','wow','sad','unhappy','horrible','regret',
                 'bad','terrible','annoyed','disappointed','upset','awful','hate']

def selectedcount(dic,word):
    if word in dic:
        return dic[word]
    else:
        return 0
    
dfwc = df.copy()  
for word in selectedwords:
    dfwc[word] = dfwc['Word Counts'].apply(selectedcount,args=(word,))
    
word_sum = dfwc[selectedwords].sum()
print('Selected Words')
print(word_sum.sort_values(ascending=False).iloc[:5])

print('\nClass Names')
print(df['Class Name'].fillna("Empty").value_counts().iloc[:5])

fig, ax = plt.subplots(1,2,figsize=(20,10))
wc0 = WordCloud(background_color='white',
                      width=450,
                      height=400 ).generate_from_frequencies(word_sum)

cn = df['Class Name'].fillna(" ").value_counts()
wc1 = WordCloud(background_color='white',
                      width=450,
                      height=400 
                     ).generate_from_frequencies(cn)

ax[0].imshow(wc0)
ax[0].set_title('Selected Words\n',size=25)
ax[0].axis('off')

ax[1].imshow(wc1)
ax[1].set_title('Class Names\n',size=25)
ax[1].axis('off')

rt = df['Review Text']
plt.subplots(figsize=(18,6))
wordcloud = WordCloud(background_color='white',
                      width=900,
                      height=300
                     ).generate(" ".join(rt))
plt.imshow(wordcloud)
plt.title('All Words in the Reviews\n',size=25)
plt.axis('off')
plt.show()

Le diagramme en barres ci-dessus montre qu'il y a un nombre maximal d'entrées pour Top, qui se situe autour de 10500. Ensuite, le service des robes compte environ 6000 entrées

In [ ]:
w=df.groupby(by=['Division Name'],as_index=False).count().sort_values(by='Class Name',ascending=False)

plt.figure(figsize=(10,10))
sns.set_style("whitegrid")
ax = sns.barplot(x=w['Division Name'],y=w['Class Name'], data=w)
plt.xlabel("Division Name")
plt.ylabel("Count")
plt.title("Counts Vs Division Name")

Au niveau de nos données, il y a 3 divisions qui sont General, General Petite et Intimates. Les produits de la division générale ont été plus vendus que les produits généraux Petite et Intimates. Environ 14 000 produits ont été vendus dans la division générale, 8 000 dans la division générale Petite et environ 1 600 produits ont été vendus dans la division Initmates.

In [ ]:
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
from wordcloud import WordCloud, STOPWORDS
import re

top_N = 100
#convert list of list into text
#a=''.join(str(r) for v in df_usa['title'] for r in v)

a = df['Review Text'].str.lower().str.cat(sep=' ')

# removes punctuation,numbers and returns list of words
b = re.sub('[^A-Za-z]+', ' ', a)

#remove all the stopwords from the text
stop_words = list(get_stop_words('en'))         
nltk_words = list(stopwords.words('english'))   
stop_words.extend(nltk_words)

word_tokens = word_tokenize(b)
filtered_sentence = [w for w in word_tokens if not w in stop_words]
filtered_sentence = []
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)

# Remove characters which have length less than 2  
without_single_chr = [word for word in filtered_sentence if len(word) > 2]

# Remove numbers
cleaned_data_title = [word for word in without_single_chr if not word.isnumeric()]        

# Calculate frequency distribution
word_dist = nltk.FreqDist(cleaned_data_title)
rslt = pd.DataFrame(word_dist.most_common(top_N),
                    columns=['Word', 'Frequency'])

plt.figure(figsize=(10,10))
sns.set_style("whitegrid")
ax = sns.barplot(x="Word",y="Frequency", data=rslt.head(7))

Le diagramme en barres ci-dessus calcule la fréquence du mot dans la colonne Texte de révision. Le mot dress est apparu plus dans le texte. A côté de cela, le mot Love vient en second, ce qui est un indicateur de révision positive.

In [ ]:
def wc(data,bgcolor,title):
    plt.figure(figsize = (100,100))
    wc = WordCloud(background_color = bgcolor, max_words = 1000,  max_font_size = 50)
    wc.generate(' '.join(data))
    plt.imshow(wc)
    plt.axis('off')

In [ ]:
wc(cleaned_data_title,'black','Most Used Words')

Le wordcloud ci-dessus concerne les mots les plus utilisés dans la colonne Texte de révision.

In [ ]:
from textblob import TextBlob

bloblist_desc = list()

df_review_str=df['Review Text'].astype(str)
for row in df_review_str:
    blob = TextBlob(row)
    bloblist_desc.append((row,blob.sentiment.polarity, blob.sentiment.subjectivity))
    df_polarity_desc = pd.DataFrame(bloblist_desc, columns = ['Review','sentiment','polarity'])
 
def f(df_polarity_desc):
    if df_polarity_desc['sentiment'] > 0:
        val = "Positive Review"
    elif df_polarity_desc['sentiment'] == 0:
        val = "Neutral Review"
    else:
        val = "Negative Review"
    return val

df_polarity_desc['Sentiment_Type'] = df_polarity_desc.apply(f, axis=1)

plt.figure(figsize=(10,10))
sns.set_style("whitegrid")
ax = sns.countplot(x="Sentiment_Type", data=df_polarity_desc)

Selon le graphique ci-dessus, il y a plus de critiques positives, mais là encore, cela dépend de la valeur de la polarité. Je pense que la valeur du sentiment > 0 est un avis positif.

In [ ]:
positive_reviews=df_polarity_desc[df_polarity_desc['Sentiment_Type']=='Positive Review']
negative_reviews=df_polarity_desc[df_polarity_desc['Sentiment_Type']=='Negative Review']

In [ ]:
wc(positive_reviews['Review'],'black','Most Used Words')

Le wordcloud ci-dessus uniquement pour les commentaires positifs.

In [ ]:
wc(negative_reviews['Review'],'black','Most Used Words')

Le wordcloud ci-dessus que pour les commentaires négatifs.

In [ ]:
import string
def text_process(review):
    nopunc=[word for word in review if word not in string.punctuation]
    nopunc=''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
df['Review Text'].head(5).apply(text_process)

Voici comment fonctionne la fonction ci-dessus. En gros, la fonction ci-dessus supprime les ponctuations, convertit les mots en minuscules et supprime les mots vides de la phrase.

## Construction d'un classificateur de sentiments:

Comme nous n'avons pas de colonne indiquant le sentiment positif ou négatif dans l'ensemble de données, j'ai défini une nouvelle colonne. Pour ce faire, j'ai supposé que les commentaires dont l'évaluation était égale ou supérieure à 4 (positive dans le nouveau cadre de données) et à l'évaluation négative les autres (faux dans le nouveau cadre de données)

In [ ]:
# Rating of 4 or higher -> positive, while the ones with 
# Rating of 2 or lower -> negative 
# Rating of 3 -> neutral
df = df[df['Rating'] != 3]
df['Sentiment'] = df['Rating'] >=4
df.head()

Avant d’ajuster les modèles, j’ai divisé les données en formation et en test. Ensuite, j'ai monté les modèles un à un pour savoir le modèle que je vais travailler avec

In [ ]:
train_data,test_data = train_test_split(df,train_size=0.8,random_state=0)
X = vectorizer.fit_transform(train_data['Review Text'])
y = train_data['Sentiment']

## Logistic Regression

In [ ]:
start=dt.datetime.now()
lr = LogisticRegression()
lr.fit(X,y)
print('Elapsed time: ',str(dt.datetime.now()-start))

## Naive Bayes

In [ ]:
start=dt.datetime.now()
nb = MultinomialNB()
nb.fit(X,y)
print('Elapsed time: ',str(dt.datetime.now()-start))

## Support Vector Machine (SVM)

In [ ]:
start=dt.datetime.now()
svm = SVC()
svm.fit(X,y)
print('Elapsed time: ',str(dt.datetime.now()-start))

## Neural Network

In [ ]:
start=dt.datetime.now()
nn = MLPClassifier()
nn.fit(X,y)
print('Elapsed time: ',str(dt.datetime.now()-start))

In [ ]:
pred_lr = lr.predict_proba(vectorizer.transform(test_data['Review Text']))[:,1]
fpr_lr,tpr_lr,_ = roc_curve(test_data['Sentiment'].values,pred_lr)
roc_auc_lr = auc(fpr_lr,tpr_lr)

pred_nb = nb.predict_proba(vectorizer.transform(test_data['Review Text']))[:,1]
fpr_nb,tpr_nb,_ = roc_curve(test_data['Sentiment'].values,pred_nb)
roc_auc_nb = auc(fpr_nb,tpr_nb)

pred_svm = svm.decision_function(vectorizer.transform(test_data['Review Text']))
fpr_svm,tpr_svm,_ = roc_curve(test_data['Sentiment'].values,pred_svm)
roc_auc_svm = auc(fpr_svm,tpr_svm)

pred_nn = nn.predict_proba(vectorizer.transform(test_data['Review Text']))[:,1]
fpr_nn,tpr_nn,_ = roc_curve(test_data['Sentiment'].values,pred_nn)
roc_auc_nn = auc(fpr_nn,tpr_nn)

f, axes = plt.subplots(2, 2,figsize=(15,10))
axes[0,0].plot(fpr_lr, tpr_lr, color='darkred', lw=2, label='ROC curve (area = {:0.2f})'.format(roc_auc_lr))
axes[0,0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[0,0].set(xlim=[-0.01, 1.0], ylim=[-0.01, 1.05])
axes[0,0].set(xlabel ='False Positive Rate', ylabel = 'True Positive Rate', title = 'Logistic Regression')
axes[0,0].legend(loc='lower right', fontsize=13)

axes[0,1].plot(fpr_nb, tpr_nb, color='darkred', lw=2, label='ROC curve (area = {:0.2f})'.format(roc_auc_nb))
axes[0,1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[0,1].set(xlim=[-0.01, 1.0], ylim=[-0.01, 1.05])
axes[0,1].set(xlabel ='False Positive Rate', ylabel = 'True Positive Rate', title = 'Naive Bayes')
axes[0,1].legend(loc='lower right', fontsize=13)

axes[1,0].plot(fpr_svm, tpr_svm, color='darkred', lw=2, label='ROC curve (area = {:0.2f})'.format(roc_auc_svm))
axes[1,0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[1,0].set(xlim=[-0.01, 1.0], ylim=[-0.01, 1.05])
axes[1,0].set(xlabel ='False Positive Rate', ylabel = 'True Positive Rate', title = 'Support Vector Machine')
axes[1,0].legend(loc='lower right', fontsize=13)

axes[1,1].plot(fpr_nn, tpr_nn, color='darkred', lw=2, label='ROC curve (area = {:0.2f})'.format(roc_auc_nn))
axes[1,1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[1,1].set(xlim=[-0.01, 1.0], ylim=[-0.01, 1.05])
axes[1,1].set(xlabel ='False Positive Rate', ylabel = 'True Positive Rate', title = 'Neural Network')
axes[1,1].legend(loc='lower right', fontsize=13)

## Vectorisation

Pour le moment, nous avons notre colonne de text revision sous forme de jeton (qui n’a pas de ponctuation ni de mots vides). CountVectorizer de Scikit-learn peut convertir la collection de textes en une matrice de comptes de jetons. Vous pouvez imaginer cette matrice résultante comme une matrice 2D, où chaque ligne est un mot unique et chaque colonne est une révision.

In [ ]:
df=df.dropna(axis=0,how='any')
rating_class = df[(df['Rating'] == 1) | (df['Rating'] == 5)]
X_review=rating_class['Review Text']
y=rating_class['Rating']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer=CountVectorizer(analyzer=text_process).fit(X_review)

In [ ]:
print(len(bow_transformer.vocabulary_))


Maintenant, le nombre ci-dessus est la taille du vocabulaire stocké dans le vectoriseur (basé sur X_review)

In [ ]:
X_review = bow_transformer.transform(X_review)

## test des donées:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_review, y, test_size=0.3, random_state=101)

## test du modèle:

Pour prédire les notes des avis, j'ai utilisé l’algorithme Naive Bayes Machine Learning. Depuis cela fonctionne bien avec les données de texte.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)

In [ ]:
predict=nb.predict(X_test)

Une fois que nous avions prédit les valeurs, la tâche la plus importante consiste maintenant à vérifier et à évaluer notre modèle par rapport aux cotes réelles (stockées dans y_test) à l'aide de confusion_matrix et classification_report de Scikit-learn.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, predict))
print('\n')
print(classification_report(y_test, predict))

Notre modèle donc a atteint une efficacité de 95%. Cela signifie que l'entreprise peut prédire si les utilisateurs ont aimé le produit ou non

## On va tester maintenat le modèle avec notre Data

In [ ]:
rating_positive=df['Review Text'][3]
rating_positive

Tout d'abord, je veux tester avec la critique positive. J'ai choisi l'examen ci-dessus et sa note est 5. Après évaluation, il devrait prédire une note de 5.

In [ ]:
rating_positive_transformed = bow_transformer.transform([rating_positive])
nb.predict(rating_positive_transformed)[0]

Deuxièmement, je veux tester avec la critique négative. J'ai choisi l'examen ci-dessous et sa note est 1. Après évaluation, il devrait prédire la note 1.

In [ ]:
rating_negative=df['Review Text'][61]
rating_negative

In [ ]:
rating_negative_transformed = bow_transformer.transform([rating_negative])
nb.predict(rating_negative_transformed)[0]

Maintenat on va prédire si un produit est recommandé ou nn?

In [ ]:
X_predict_recommend=df['Review Text']
y_recommend=df['Recommended IND']

bow_transformer=CountVectorizer(analyzer=text_process).fit(X_predict_recommend)

X_predict_recommend = bow_transformer.transform(X_predict_recommend)

X_train, X_test, y_train, y_test = train_test_split(X_predict_recommend, y_recommend, test_size=0.3, random_state=101)

nb = MultinomialNB()
nb.fit(X_train, y_train)

predict_recommendation=nb.predict(X_test)


print(confusion_matrix(y_test, predict_recommendation))
print('\n')
print(classification_report(y_test, predict_recommendation))

In [ ]:
rating_positive

In [ ]:
rating_positive_transformed = bow_transformer.transform([rating_positive])
nb.predict(rating_positive_transformed)[0]

In [ ]:
rating_negative

In [ ]:
rating_negative_transformed = bow_transformer.transform([rating_negative])
nb.predict(rating_negative_transformed)[0]